# Imports

In [1]:
#Library to make GUI
from tkinter import messagebox
import tkinter as tk

import requests 
from bs4 import BeautifulSoup

import pandas as pd
import numpy as np

import spotipy # install if needed
from spotipy.oauth2 import SpotifyClientCredentials

import getpass

from fuzzywuzzy import fuzz

# Get Data from Billboard to know de 100 hot songs

In [2]:
# Create URL
url="https://www.billboard.com/charts/hot-100/"

# Create a request
results=requests.get(url, headers = {"Accept-Language": "en-US"})

# Result in string
#results.text

# We use BeautifulSoup to extract specific results
soup=BeautifulSoup(results.content,'html.parser')

# Show results in html mode
#print(soup.prettify())

# Strip=True => You can tell Beautiful Soup to strip whitespace from the beginning and end of each bit of text
# Try get the first name of song
soup.select('h3.c-title.a-no-trucate')[0].get_text(strip=True)

# Try get the first artist of song
soup.select('span.c-label.a-no-trucate')[0].get_text(strip=True)

# Calculate the len of our select
len_songs=len(soup.select('h3.c-title.a-no-trucate'))

# Create variables to store results
song = []
artist = []

# Create a for to pick the 100 best songs
for x in range(len_songs):
    song.append(soup.select('h3.c-title.a-no-trucate')[x].get_text(strip=True))
    artist.append(soup.select('span.c-label.a-no-trucate')[x].get_text(strip=True))
    
# Import pandas to create a data frame
top=pd.DataFrame({'Song':song, 'Artist':artist})
#top

# Import numpy to add a Rank
top['Rank']=np.arange(len(top))+1
top


,Song,Artist,Rank
0,We Don't Talk About Bruno,"Carolina Gaitan, Mauro Castillo, Adassa, Rhenz...",1
1,Heat Waves,Glass Animals,2
2,abcdefu,GAYLE,3
3,Easy On Me,Adele,4
4,Super Gremlin,Kodak Black,5
...,...,...,...
95,Closer,Saweetie Featuring H.E.R.,96
96,Cold December,Rod Wave,97
97,Get Into It (Yuh),Doja Cat,98
98,All Too Well (Taylor's Version),Taylor Swift,99


# Get playlist from Spotify to analice audio features

In [3]:
# Get usser and pass
client_id=str(getpass.getpass('client_id?'))
client_secret=str(getpass.getpass('client_secret?'))

# Create a conexion
sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(client_id=client_id, client_secret=client_secret))

# Get random playlist
playlist=sp.user_playlist_tracks("Spotify","37i9dQZF1DX4WYpdgoIcn6")
playlist

client_id?········
client_secret?········


{'href': 'https://api.spotify.com/v1/playlists/37i9dQZF1DX4WYpdgoIcn6/tracks?offset=0&limit=100&additional_types=track',
 'items': [{'added_at': '2022-02-24T23:02:00Z',
   'added_by': {'external_urls': {'spotify': 'https://open.spotify.com/user/'},
    'href': 'https://api.spotify.com/v1/users/',
    'id': '',
    'type': 'user',
    'uri': 'spotify:user:'},
   'is_local': False,
   'primary_color': None,
   'track': {'album': {'album_type': 'album',
     'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/06HL4z0CvFAxyc27GXpf02'},
       'href': 'https://api.spotify.com/v1/artists/06HL4z0CvFAxyc27GXpf02',
       'id': '06HL4z0CvFAxyc27GXpf02',
       'name': 'Taylor Swift',
       'type': 'artist',
       'uri': 'spotify:artist:06HL4z0CvFAxyc27GXpf02'}],
     'available_markets': ['AD',
      'AE',
      'AG',
      'AL',
      'AM',
      'AO',
      'AR',
      'AT',
      'AU',
      'AZ',
      'BA',
      'BB',
      'BD',
      'BE',
      'BF',
      'BG'

In [4]:
# As we have limit 100, need do while to get all songs of the play list
# While playlist have next we can get new data
def get_tracks(playlist):
    tracks = playlist['items']
    while playlist['next']:
        playlist = sp.next(playlist)
        tracks.extend(playlist['items'])
    return tracks
first_results = get_tracks(playlist)

# get track list with url uri or ID for each track - ID in this case
audio_feats = []
for x in range(len(first_results)):
    audio_feats.extend(sp.audio_features(tracks=first_results[x]['track']['id']))

# Transform in to Dataframe
data = pd.DataFrame(audio_feats)
data

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature
0,0.392,0.574,7,-9.195,1,0.1700,0.833,0.001790,0.1450,0.5290,81.112,audio_features,0lx2cLdOt3piJbcaXIV74f,spotify:track:0lx2cLdOt3piJbcaXIV74f,https://api.spotify.com/v1/tracks/0lx2cLdOt3pi...,https://api.spotify.com/v1/audio-analysis/0lx2...,214707,4
1,0.604,0.366,5,-7.519,1,0.0282,0.578,0.000000,0.1330,0.1300,141.981,audio_features,0gplL1WMoJ6iYaPgMCL0gX,spotify:track:0gplL1WMoJ6iYaPgMCL0gX,https://api.spotify.com/v1/tracks/0gplL1WMoJ6i...,https://api.spotify.com/v1/audio-analysis/0gpl...,224695,4
2,0.700,0.613,11,-4.041,1,0.0408,0.385,0.000000,0.0887,0.4200,88.084,audio_features,7eQHxigpuDJjCG50JyzU8v,spotify:track:7eQHxigpuDJjCG50JyzU8v,https://api.spotify.com/v1/tracks/7eQHxigpuDJj...,https://api.spotify.com/v1/audio-analysis/7eQH...,191250,4
3,0.422,0.340,0,-6.577,1,0.0351,0.877,0.000018,0.1490,0.2230,77.213,audio_features,1is8gU4RVcN4J8xItxWoOY,spotify:track:1is8gU4RVcN4J8xItxWoOY,https://api.spotify.com/v1/tracks/1is8gU4RVcN4...,https://api.spotify.com/v1/audio-analysis/1is8...,204094,4
4,0.516,0.467,0,-6.184,1,0.0302,0.662,0.000000,0.0853,0.3860,132.861,audio_features,5WtfUKzXircvW8l5aaVZWT,spotify:track:5WtfUKzXircvW8l5aaVZWT,https://api.spotify.com/v1/tracks/5WtfUKzXircv...,https://api.spotify.com/v1/audio-analysis/5Wtf...,173307,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
125,0.517,0.435,9,-7.532,1,0.0278,0.400,0.000000,0.1230,0.3350,97.931,audio_features,5Tl8bvgo5shLODXiT9ckva,spotify:track:5Tl8bvgo5shLODXiT9ckva,https://api.spotify.com/v1/tracks/5Tl8bvgo5shL...,https://api.spotify.com/v1/audio-analysis/5Tl8...,200220,4
126,0.535,0.314,7,-12.823,0,0.0408,0.895,0.000150,0.0874,0.0663,145.095,audio_features,27Y1N4Q4U3EfDU5Ubw8ws2,spotify:track:27Y1N4Q4U3EfDU5Ubw8ws2,https://api.spotify.com/v1/tracks/27Y1N4Q4U3Ef...,https://api.spotify.com/v1/audio-analysis/27Y1...,187601,4
127,0.533,0.500,7,-8.273,1,0.0481,0.772,0.000000,0.1120,0.1400,99.770,audio_features,38ovefcipRlByRd5UF9PWp,spotify:track:38ovefcipRlByRd5UF9PWp,https://api.spotify.com/v1/tracks/38ovefcipRlB...,https://api.spotify.com/v1/audio-analysis/38ov...,178143,4
128,0.824,0.783,1,-4.530,0,0.0507,0.220,0.000000,0.2340,0.7880,114.028,audio_features,3xvYvJAa0iUVKKsYcuDNlK,spotify:track:3xvYvJAa0iUVKKsYcuDNlK,https://api.spotify.com/v1/tracks/3xvYvJAa0iUV...,https://api.spotify.com/v1/audio-analysis/3xvY...,160825,4


# Show results with GUI (Tkinter)

In [5]:
# Create a new window
def retry(window):
    window.destroy()
    create()

# Close window
def close(window):
    window.destroy()

# Show the results after compare input and top[song] when the match is positive
# Check has the Dataframe with 1 positive result
def but_yes(window,check,top):
    # Create a new dataframe without result to avoid repeat
    result = top.drop(labels=check.index, axis=0)

    # Create a new dataframe with a random sample to share
    result = result.sample()
    song_res = result['Song'].item()
    artist_res = result['Artist'].item()

    # Change size os window
    window.geometry("")
    # Show the results
    res1 = tk.Label(window, text = 'Nice choice, your song is in the top 100, maybe you like this one too')
    res1.pack(fill = tk.X)
    res2 = tk.Label(window, text = f'Song:{song_res}- Artist:{artist_res}')
    res2.pack(fill = tk.X)
    space = tk.Label(window, text = "   ")
    space.pack(fill = tk.X)
    
    #  Ask for Try Again
    label_no = tk.Label(window, text = 'Do you want to try again?')
    label_no.pack(fill = tk.X)
    button_retry = tk.Button(window, text = 'Try again', command = lambda: retry(window))
    button_retry.pack()
    button_close = tk.Button(window, text = 'No', command = lambda: close(window))
    button_close.pack()

#If the match is negative, show the result and ask for Try Again
def but_no(window):
    label_no = tk.Label(window, text = f'Sorry, I dont know what song you mean. Do you want to try again?')
    label_no.pack(fill = tk.X)
    button_retry = tk.Button(window, text = 'Try again', command = lambda: retry(window))
    button_retry.pack()
    button_close = tk.Button(window, text = 'No', command = lambda: close(window))
    button_close.pack()

# Get data from inputs
def get_data(input_song,top,window):
    # Transform to string
    songs = input_song.get()
    if len(songs) == 0:
        messagebox.showwarning('Error','You need to enter at least one song.')
    else:        
        # fuzz token compare the input with the songs of the dataframe
        # we save in -best- the best result
        res = 0
        for x in top['Song']:
            if fuzz.token_set_ratio(x,songs) > res:
                res = fuzz.token_set_ratio(x,songs)
                best = x

        # get data from best result
        check = top[(top['Song']==best)]
        artist = check['Artist'].item()

        # Show the results
        window.geometry("")
        ask = tk.Label(window, text = f'Did you mean this song? ->  {best} - {artist}')
        ask.pack(fill = tk.X)
    
        button_yes = tk.Button(window, text = 'Yes', command = lambda: but_yes(window,check,top))
        button_yes.pack()
        button_no = tk.Button(window, text = 'No', command = lambda: but_no(window))
        button_no.pack()
        space = tk.Label(window, text = "   ")
        space.pack(fill = tk.X)

def create():
    
    # Create a window
    window = tk.Tk()

    #Change size os window
    window.geometry("")

    # Title
    title = tk.Label(window, text = 'Enter a song and/or an artist to see if its in the top 100', bg = 'gray51')
    title.pack(fill = tk.X)

    #Take song
    title_song = tk.Label(window, text = 'Enter a song')
    title_song.pack(fill = tk.X)
    input_song = tk.Entry(window)
    input_song.pack()

    button_search = tk.Button(window, text = 'Search', command = lambda: get_data(input_song,top,window))
    button_search.pack()
    space = tk.Label(window, text = "   ")
    space.pack(fill = tk.X)
    
    #That comand open the window
    window.mainloop()

In [8]:
create()